<a href="https://colab.research.google.com/github/ashishpal2702/HumanActivityrecognition/blob/main/Logistic_Regression_and_Classification_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

We will be using the [Human Activity Recognition with Smartphones](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) database, which was built from the recordings of study participants performing activities of daily living (ADL) while carrying a smartphone with an embedded inertial sensors. The objective is to classify activities into one of the six activities (walking, walking upstairs, walking downstairs, sitting, standing, and laying) performed.

For each record in the dataset it is provided: 

- Triaxial acceleration from the accelerometer (total acceleration) and the estimated body acceleration. 
- Triaxial Angular velocity from the gyroscope. 
- A 561-feature vector with time and frequency domain variables. 
- Its activity label. 

More information about the features is available on the website above.

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
from __future__ import print_function
import os
data_path = [ 'data']

from sklearn.preprocessing import LabelEncoder , StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import ExtraTreesClassifier


In [3]:
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

## 1. Data Import

Import the data and do the following:

* Examine the data types--there are many columns, so it might be wise to use value counts
* Determine if the floating point values need to be scaled
* Determine the breakdown of each activity
* Encode the activity label as an integer

In [4]:
import pandas as pd
import numpy as np
import os
#filepath = '/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data//Human_Activity_Recognition_Using_Smartphones_Data_augmented_data.gzip'
filepath = '/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data/Human_Activity_Recognition_Using_Smartphones_Data_augmented_timedata_new_data.gzip'
data = pd.read_parquet(filepath)

In [5]:
data.shape

(100000, 563)

In [6]:
sensors = set()
for col in data.columns:
    sensors.add(col.split("-")[0])

In [7]:
sensors

{'Activity',
 'angle(X,gravityMean)',
 'angle(Y,gravityMean)',
 'angle(Z,gravityMean)',
 'angle(tBodyAccJerkMean),gravityMean)',
 'angle(tBodyAccMean,gravity)',
 'angle(tBodyGyroJerkMean,gravityMean)',
 'angle(tBodyGyroMean,gravityMean)',
 'date_time',
 'fBodyAcc',
 'fBodyAccJerk',
 'fBodyAccMag',
 'fBodyBodyAccJerkMag',
 'fBodyBodyGyroJerkMag',
 'fBodyBodyGyroMag',
 'fBodyGyro',
 'tBodyAcc',
 'tBodyAccJerk',
 'tBodyAccJerkMag',
 'tBodyAccMag',
 'tBodyGyro',
 'tBodyGyroJerk',
 'tBodyGyroJerkMag',
 'tBodyGyroMag',
 'tGravityAcc',
 'tGravityAccMag'}

In [8]:
data['Activity'].value_counts()

Activity
LAYING                16762
WALKING               16728
WALKING_UPSTAIRS      16675
STANDING              16645
WALKING_DOWNSTAIRS    16627
SITTING               16563
Name: count, dtype: int64

In [9]:
data.describe().T

,count,mean,min,25%,50%,75%,max,std
tBodyAcc-mean()-X,100000.0,0.138041,-0.86304,0.066226,0.134419,0.205561,0.771421,0.08652
tBodyAcc-mean()-Y,100000.0,-0.008688,-0.584658,-0.01382,-0.00717,-0.002103,0.511605,0.017784
tBodyAcc-mean()-Z,100000.0,-0.054425,-0.918692,-0.080716,-0.051144,-0.024272,0.937914,0.040249
tBodyAcc-std()-X,100000.0,-0.282349,-0.998199,-0.492172,-0.197145,-0.045912,0.839073,0.307097
tBodyAcc-std()-Y,100000.0,-0.233839,-0.994999,-0.465923,-0.118005,-0.005036,0.934541,0.320421
...,...,...,...,...,...,...,...,...
"angle(tBodyGyroJerkMean,gravityMean)",100000.0,-0.006645,-0.931989,-0.128887,-0.001805,0.112867,0.962053,0.245722
"angle(X,gravityMean)",100000.0,-0.261718,-0.992246,-0.50046,-0.281813,-0.067636,0.948844,0.319036
"angle(Y,gravityMean)",100000.0,0.039957,-0.975458,0.003913,0.060276,0.138061,0.988527,0.170272
"angle(Z,gravityMean)",100000.0,-0.020835,-0.944995,-0.042101,0.000425,0.03913,0.991536,0.149135


In [10]:
le = LabelEncoder()
data['Activity'] = le.fit_transform(data['Activity'])

In [11]:
data['Activity'].value_counts()

Activity
0    16762
3    16728
5    16675
2    16645
4    16627
1    16563
Name: count, dtype: int64

### Feature engineering

In [12]:
from sklearn.model_selection import train_test_split
X = data.drop(['date_time','Activity'] , axis = 1)
Y = data['Activity']

In [13]:
def get_top_k_features(X, Y, k):
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(X, Y)
    feature_df = pd.DataFrame(data=(X.columns, clf.feature_importances_)).T.sort_values(by=1, ascending=False)
    cols = feature_df.head(k)[0].values
    return cols

In [14]:
top_k_features = get_top_k_features(X, Y, k=10)


In [15]:
top_k_features


array(['tGravityAcc-energy()-X', 'tGravityAcc-max()-X',
       'tGravityAcc-min()-X', 'angle(X,gravityMean)',
       'tGravityAcc-mean()-X', 'tGravityAcc-max()-Y',
       'tGravityAcc-mean()-Y', 'angle(Y,gravityMean)', 'tBodyAcc-max()-X',
       'tGravityAcc-min()-Y'], dtype=object)

## 3. Data preparation

* Split the data into train and test data sets. 
* Regardless of methods used to split the data, compare the ratio of classes in both the train and test splits.


In [16]:
X = X[top_k_features]

In [17]:
x_train , x_test , y_train , y_test = train_test_split(X, Y)


In [18]:
x_train.shape , y_train.shape


((75000, 10), (75000,))

In [19]:
x_test.shape , y_test.shape


((25000, 10), (25000,))

## Standardise data

In [20]:
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.transform(x_test)


## 4. Model Training

* Fit different models and compare the result 
1. Logistic regression
2. Decision Tree Classifier
3. Random Forest Classifier 
4. Adaptive Boosting Classifier

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [22]:
model_result = {}


In [23]:
lr =  LogisticRegression()
lr.fit(x_train_std, y_train)
train_accuracy = round(lr.score(x_train_std, y_train)*100,2)
test_accuracy = round(lr.score(x_test_std, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['Logistic_Regression'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }


Training Accuracy 72.17
Test Accuracy 72.07


/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/code/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
dt =  DecisionTreeClassifier()
dt.fit(x_train, y_train)
train_accuracy = round(dt.score(x_train, y_train)*100,2)
test_accuracy = round(dt.score(x_test, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['Decision_Tree'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }


Training Accuracy 100.0
Test Accuracy 65.58


In [25]:
rfc =  RandomForestClassifier(n_estimators=20)
rfc.fit(x_train, y_train)
train_accuracy = round(rfc.score(x_train, y_train)*100,2)
test_accuracy = round(rfc.score(x_test, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['RandomForest'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }


Training Accuracy 99.76
Test Accuracy 73.19


In [26]:
gbc =  GradientBoostingClassifier(n_estimators=20)
gbc.fit(x_train, y_train)
train_accuracy = round(gbc.score(x_train, y_train)*100,2)
test_accuracy = round(gbc.score(x_test, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)
model_result['GradientBoosting'] = {'train_accuracy': train_accuracy,'test_accuracy':test_accuracy }

Training Accuracy 73.1
Test Accuracy 72.71


## 5. Model comparison

In [27]:
pd.DataFrame(model_result).T

,train_accuracy,test_accuracy
Logistic_Regression,72.17,72.07
Decision_Tree,100.00,65.58
RandomForest,99.76,73.19
GradientBoosting,73.10,72.71


## Hyper parameter tuning

In [28]:
from sklearn.model_selection import GridSearchCV
param_grid = {
            "n_estimators": [
                10,
                50,
                100,
            ],
            "max_depth": [4, 12],
            "criterion": ["gini", "entropy"],
        }
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(n_estimators=20),
             param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [4, 12],
                         'n_estimators': [10, 50, 100]})

In [29]:
CV_rfc.best_params_

{'criterion': 'gini', 'max_depth': 12, 'n_estimators': 100}

In [30]:
rfc =  RandomForestClassifier(criterion= 'entropy', max_depth= 12, n_estimators= 100)
rfc.fit(x_train, y_train)
train_accuracy = round(rfc.score(x_train, y_train)*100,2)
test_accuracy = round(rfc.score(x_test, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)

Training Accuracy 76.92
Test Accuracy 74.27


In [31]:
train_accuracy = round(CV_rfc.score(x_train, y_train)*100,2)
test_accuracy = round(CV_rfc.score(x_test, y_test)*100,2)
print("Training Accuracy", train_accuracy)
print("Test Accuracy", test_accuracy)

Training Accuracy 76.9
Test Accuracy 74.12


### 6. Final Model

In [32]:
importances = rfc.feature_importances_

In [33]:
## Top 10 Features contributing to Model 
feature_df = pd.DataFrame( x_train.columns ,  columns = ['variables'])
feature_df['importance'] = importances
feature_df.sort_values( by = 'importance' , ascending = False).head(10)


,variables,importance
8,tBodyAcc-max()-X,0.354443
2,tGravityAcc-min()-X,0.139880
4,tGravityAcc-mean()-X,0.098670
9,tGravityAcc-min()-Y,0.087574
3,"angle(X,gravityMean)",0.069273
6,tGravityAcc-mean()-Y,0.064700
5,tGravityAcc-max()-Y,0.052039
7,"angle(Y,gravityMean)",0.049416
1,tGravityAcc-max()-X,0.046727
0,tGravityAcc-energy()-X,0.037277


In [34]:
y_pred = rfc.predict(x_test)

## 7. Model Evaluation

For each model, calculate the following error metrics: 

* accuracy
* precision
* recall
* fscore
* confusion matrix

Decide how to combine the multi-class metrics into a single value for each model.

In [35]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report


In [36]:
confusion_matrix(y_pred, y_test)

array([[4195,    0,    0,    0,    0,    0],
       [   0, 3554,  468,  143,   17,  122],
       [   0,  394, 2722,  359,    4,   64],
       [   0,  207,  657, 1968,  202,  800],
       [   0,    0,    0,  641, 3623,  653],
       [   0,   19,  240, 1080,  362, 2506]])

In [37]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4195
           1       0.85      0.83      0.84      4304
           2       0.67      0.77      0.71      3543
           3       0.47      0.51      0.49      3834
           4       0.86      0.74      0.79      4917
           5       0.60      0.60      0.60      4207

    accuracy                           0.74     25000
   macro avg       0.74      0.74      0.74     25000
weighted avg       0.75      0.74      0.75     25000



## 8. Model registration


In [38]:
## Change Drive path to your Folder
#os.mkdir('model_weights/')
#os.mkdir('model_features/')

In [39]:
## Let's save model weights
import joblib
# save
joblib.dump(rfc, "./model_weights/my_random_forest.joblib")

['./model_weights/my_random_forest.joblib']

In [40]:
## Save Final columns
final_columns = np.array(x_train.columns) 
joblib.dump(final_columns, "./model_features/train_features.joblib")

['./model_features/train_features.joblib']

## 9. Model Prediction

In [41]:
## load Test Data set 
test_data = pd.read_csv('/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data/Human_Activity_Recognition_Using_Smartphones_TestData.csv')

In [42]:
## load Test Data set 
test_data = pd.read_csv('/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data/test_data_100.csv')

In [43]:
test_data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,STANDING


In [44]:
## Load Features and model weight
train_features = joblib.load("./model_features/train_features.joblib")

model = joblib.load("./model_weights/my_random_forest.joblib")


In [45]:
test_data_features = test_data[train_features]

In [47]:
test_data_features.fillna(0,inplace = True)

/var/folders/h7/7cnm33m55zqb3ckcp7485gfw0000gn/T/ipykernel_17498/237628222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_features.fillna(0,inplace = True)


In [48]:
y_prediction = model.predict(test_data_features)
y_prediction_label = le.inverse_transform(y_prediction)
test_data['Prediction_label'] = y_prediction_label

In [49]:
test_data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",Activity,Prediction_label
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,STANDING,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,STANDING,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,STANDING,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,STANDING,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,STANDING,STANDING


In [50]:
test_data['Prediction_label'].value_counts()

Prediction_label
STANDING    30
LAYING      27
SITTING     24
WALKING     19
Name: count, dtype: int64

#### Evaluate Model prediction

In [51]:
y_test = test_data['Activity'].values
y_pred = test_data['Prediction_label']

In [52]:
confusion_matrix(y_pred, y_test)

array([[27,  0,  0,  0],
       [ 0, 24,  0,  0],
       [ 0,  0, 27,  3],
       [ 0,  0,  0, 19]])

In [53]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00        27
     SITTING       1.00      1.00      1.00        24
    STANDING       1.00      0.90      0.95        30
     WALKING       0.86      1.00      0.93        19

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100



## End

In [ ]:
v1_df = pd.read_parquet('/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data/data_drift/v1.gzip')
v2_df = pd.read_parquet('/Users/apal/Documents/PathtoAI/AnalyticsVidhya/Mlops/data/data_drift/v2.gzip')